In [1]:
from bs4 import BeautifulSoup
import html.parser
import json
import csv
import time
import re
import requests

In [2]:
urls = []
with open('urls_90.csv', 'r' ) as csv_file:
    csv_reader = csv.reader(csv_file)
    for line in csv_reader:
        url = str(line[0]).strip()
        urls.append(url)
print(len(urls))

6119


In [8]:
print(urls[13])

https://www.e-jumbo.gr/axesouar-i-y-kiniton/kalodia-fortistes-kiniton-tablet/fortistes-kiniton-tablet/polyfortistis-taxidiou-all-in-1-2usb-thyres_307979/


In [6]:
for i in range(0,len(urls)):
    url = urls[i]
    headers = {
        'authority': 'www.e-jumbo.gr',
        'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
        'accept': 'application/json, text/javascript, */*; q=0.01',
        'x-requested-with': 'XMLHttpRequest',
        'sec-ch-ua-mobile': '?0',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36',
        'content-type': 'application/json; charset=UTF-8',
        'origin': 'https://www.e-jumbo.gr',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'cors',
        'sec-fetch-dest': 'empty',
        'referer': url,
        'accept-language': 'en-US,en;q=0.9,hi;q=0.8',
    }

    params = (
        ('lang', 'el'),
    )


    r_1 = requests.get(url, headers=headers)
    soup = BeautifulSoup(r_1.text, 'html.parser')

    try:
        name = str(soup.select_one('h1[itemprop="name"]').text).strip()
    except:
        pass

    try:
        price = str(soup.select_one('[class="current-price"]').text).strip()
    except:
        price = ""

    try:    
        sku  = str(soup.select_one('[class="code"]').text).replace("Κωδικός Jumbo:","").strip()
    except:
        sku  = ""

    try:
        category = ""
        cats = soup.select('[class="breadcrumb clearfix"] [class="title"]')
        for cat in cats[:4]:
            category += str(cat.text).strip() + " > "
        category = category.strip(" > ")
    except:
        category = ""

    try:
        images = ""
        images_raw=soup.select('a[class="cloud-zoom"]')
        for img in images_raw:
            images += "https:" + str(img["href"]).strip().split(".jpg?")[0] + ".jpg" + "\n"
        images = images.strip("\n") 
        #print(images)
    except:
        images = ""

    try:
        description = ""
        sections = soup.select('[class="tabs-content"] [class*="tab-box"]')
        for sec in sections:
            label = str(sec.select_one('[class="title"]').text).strip()
            if label =="Περιγραφή - Χαρακτηριστικά":
                description = str(sec.select_one('[class="dynamic-content"]').text).strip()
    except:
        description = ""

    f_data = [url,name,description,"","","",images,price,category]

    # with open('output_90.csv', 'a', newline='', encoding="utf-8", errors="ignore") as new_file:
    #     csv_writer = csv.writer(new_file)
    #     csv_writer.writerow(f_data)    



    skus = str(soup.select_one('[class*="ProductAttributes_ExtraSearchText"] [class*="att-cell-value"]').text).strip().split(" ")

    versions = soup.select('[class="versions-container"] [class*="x-product-details-version-selector"] li')

    skuIds = []
    sizes  = []
    for version in versions:
        size  = str(version.select_one('[class="size"]').text).strip()
        sizes.append(size)
        skuId = str(version["data-value"]).strip()
        skuIds.append(skuId)

    if len(versions)==0:
        #sku   = str(soup.select_one('[id="search-warehouse-wrapper"]')["data-skucode"]).strip()
        skuId = str(soup.select_one('[id="search-warehouse-wrapper"]')["data-skuid"]).strip()
        skuIds.append(skuId)
        sizes.append("None")
    #print(skus)
    #print(skuIds)
    #print(sizes)
    #print()
    
    for j in range(len(skus)):
        
        sku   = skus[j]
        skuId = skuIds[j]
        size  = sizes[j]

        data = {"skuId":int(skuId),"skuCode":int(sku),"countryCode":"CY","zipCode":"","maxDistance":1000000,"ignoreAvailability":'true',"geoLocationWithZipCode":'true',"topAvailableWarehouse":100000,"hasGeolocation":'false',"latitude":0,"longitude":0}
        data = json.dumps(data)

        r_2 = requests.post('https://www.e-jumbo.gr/services/EcomService.svc/GetProductAvailableWarehouses', headers=headers, params=params, data=data)
        w_data = json.loads(r_2.json()["d"]["data"])

        availability = "Μη διαθέσιμο"

        try:
            warehouses   = w_data["WarehousesGrouped"][0]["Items"]
        except:
            warehouses   =  []
            #availability = str(soup.select_one('[class*="availability-text"]').text).strip()

        for wh in warehouses:
            area = str(wh["Area"]).strip()
            if area == "Λάρνακα":
                availability = str(wh["AvailabilityName"]).strip()

        f_data = ["","","",sku,size,availability,"","",""]

        with open('output_90.csv', 'a', newline='', encoding="utf-8", errors="ignore") as new_file:
            csv_writer = csv.writer(new_file)
            csv_writer.writerow(f_data)

        print(i,r_2.status_code)

['1200357']
['1502512']
['None']

['1200356']
['1502511']
['None']

['1193617']
['1478215']
['None']

['1201470']
['1506321']
['None']

['1147687']
['275064']
['None']

['1172284']
['1373339']
['None']

['1166267']
['1355661']
['None']

['1150306']
['278547']
['None']

['1172291']
['1373420']
['None']

['1188108']
['1437158']
['None']

['1180370']
['1394849']
['None']



KeyboardInterrupt: 

In [ ]:
data = {"skuId":int(skuId),"skuCode":int(sku),"countryCode":"CY","zipCode":"","maxDistance":1000000,"ignoreAvailability":'true',"geoLocationWithZipCode":'true',"topAvailableWarehouse":100000,"hasGeolocation":'false',"latitude":0,"longitude":0}
    data = json.dumps(data)
    #print(data)

    r_2 = requests.post('https://www.e-jumbo.gr/services/EcomService.svc/GetProductAvailableWarehouses', headers=headers, params=params, data=data)
    w_data = json.loads(r_2.json()["d"]["data"])

    availability = "Μη διαθέσιμο" 

    try:
        warehouses   = w_data["WarehousesGrouped"][0]["Items"]
    except:
        warehouses   =  []
        availability = "διαθέσιμη στο διαδίκτυο"

    for wh in warehouses:
        area = str(wh["Area"]).strip()
        if area == "Λάρνακα":
            availability = str(wh["AvailabilityName"]).strip()

    f_data = ["","","","",size,availability,"","",""]

    with open('output_90.csv', 'a', newline='', encoding="utf-8", errors="ignore") as new_file:
        csv_writer = csv.writer(new_file)
        csv_writer.writerow(f_data)

    print(i,r_1.status_code,r_2.status_code)